Skeleton Code

In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time
from sklearn.naive_bayes import GaussianNB

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.


<ipython-input-1-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-1-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [14]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [15]:
df.shape

(7352, 561)

In [16]:
df.describe

<bound method NDFrame.describe of            0         1         2         3         4         5         6    \
0     0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
1     0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
2     0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
3     0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
4     0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   
...        ...       ...       ...       ...       ...       ...       ...   
7347  0.299665 -0.057193 -0.181233 -0.195387  0.039905  0.077078 -0.282301   
7348  0.273853 -0.007749 -0.147468 -0.235309  0.004816  0.059280 -0.322552   
7349  0.273387 -0.017011 -0.045022 -0.218218 -0.103822  0.274533 -0.304515   
7350  0.289654 -0.018843 -0.158281 -0.219139 -0.111412  0.268893 -0.310487   
7351  0.351503 -0.012423 -0.203867 -0.269270 -0.087212  0.177404 -0.377404   

           7         8         9    ...       551       552       553  \
0    -0.983185 -0.923527 -0.934724  ... -0.074323 -0.298676 -0.710304   
1    -0.974914 -0.957686 -0.943068  ...  0.158075 -0.595051 -0.861499   
2    -0.963668 -0.977469 -0.938692  ...  0.414503 -0.390748 -0.760104   
3    -0.982750 -0.989302 -0.938692  ...  0.404573 -0.117290 -0.482845   
4    -0.979672 -0.990441 -0.942469  ...  0.087753 -0.351471 -0.699205   
...        ...       ...       ...  ...       ...       ...       ...   
7347  0.043616  0.060410  0.210795  ... -0.070157 -0.588433 -0.880324   
7348 -0.029456  0.080585  0.117440  ...  0.165259 -0.390738 -0.680744   
7349 -0.098913  0.332584  0.043999  ...  0.195034  0.025145 -0.304029   
7350 -0.068200  0.319473  0.101702  ...  0.013865  0.063907 -0.344314   
7351 -0.038678  0.229430  0.269013  ... -0.058402 -0.387052 -0.740738   

           554       555       556       557       558       559       560  
0    -0.112754  0.030400 -0.464761 -0.018446 -0.841247  0.179941 -0.058627  
1     0.053477 -0.007435 -0.732626  0.703511 -0.844788  0.180289 -0.054317  
2    -0.118559  0.177899  0.100699  0.808529 -0.848933  0.180637 -0.049118  
3    -0.036788 -0.012892  0.640011 -0.485366 -0.848649  0.181935 -0.047663  
4     0.123320  0.122542  0.693578 -0.615971 -0.847865  0.185151 -0.043892  
...        ...       ...       ...       ...       ...       ...       ...  
7347 -0.190437  0.829718  0.206972 -0.425619 -0.791883  0.238604  0.049819  
7348  0.064907  0.875679 -0.879033  0.400219 -0.771840  0.252676  0.050053  
7349  0.052806 -0.266724  0.864404  0.701169 -0.779133  0.249145  0.040811  
7350 -0.101360  0.700740  0.936674 -0.589479 -0.785181  0.246432  0.025339  
7351 -0.280088 -0.007739 -0.056088 -0.616956 -0.783267  0.246809  0.036695  

[7352 rows x 561 columns]>

In [17]:
df.isnull().sum()

,0
0,0
1,0
2,0
3,0
4,0
...,...
556,0
557,0
558,0
559,0


In [18]:
df.nunique()

,0
0,7347
1,7352
2,7349
3,7349
4,7351
...,...
556,7352
557,7352
558,7352
559,7352


In [19]:
# Task 2: Encode class labels
# YOUR CODE HERE: Use LabelEncoder to encode class labels

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())

In [20]:
# Task 3: Scale the features using StandardScaler
# YOUR CODE HERE: Apply StandardScaler to df

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [21]:
# Task 4: Split the data into training and testing sets
# YOUR CODE HERE: Use train_test_split to split the data
X_train_full, X_test_full, y_train, y_test = train_test_split(df_scaled, encoded_y,test_size = 0.2, random_state = 42)

In [22]:
#TASK 5 - 1. Create a pipeline using Gaussian Naive Bayes
#         2. Fit the model to the training data
#         3. Predict values for test set
#         4. Print accuracy score
#TASK 6 - 1. Note the start time before defining the pipeline
#         2. Note the end time and report the difference as the time taken by the model training and inference.
start_time = time.time()
classifier_pipeline_full = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_full.fit(X_train_full, y_train)
y_pred_full = classifier_pipeline_full.predict(X_test_full)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(y_test, y_pred_full)


In [26]:
# TASK 7 - K-Means for dimensionality reduction
n_clusters = 50
kmeans = KMeans(n_clusters, random_state=42, n_init = 10)
kmeans.fit(df_scaled.T)# Transpose to treat features as data points
closest_clusters = kmeans.labels_
selected_features_indices = [np.random.choice(np.where(kmeans.labels_==i)[0]) for i in range(n_clusters)]
selected_features = df.iloc[:,selected_features_indices]

In [28]:
#TASK 8 - Train another model (GaussianNB) on the new dataset, and report time taken and accuracy

selected_features_indices = [np.random.choice(np.where(kmeans.labels_==i)[0]) for i in range(n_clusters)]
selected_features = df.iloc[:,selected_features_indices]
X_train, X_test, y_train, y_test = train_test_split(selected_features, encoded_y, test_size=0.2, random_state=42)
scaler_reduced = StandardScaler()
X_train = scaler_reduced.fit_transform(X_train)
X_test = scaler_reduced.transform(X_test)
start_time_reduced = time.time()
classifier_pipeline_reduced = Pipeline([('classifier', GaussianNB())])
classifier_pipeline_reduced.fit(X_train, y_train)
y_pred_reduced = classifier_pipeline_reduced.predict(X_test)
end_time_reduced = time.time()
reduced_features_time = end_time_reduced - start_time_reduced
accuracy_reduced = accuracy_score(y_test, y_pred_reduced)
print(f"Time taken with reduced features: {reduced_features_time:.4f} seconds")
print(f"Accuracy with reduced features: {accuracy_reduced:.4f}")





Time taken with reduced features: 0.0092 seconds
Accuracy with reduced features: 0.8042
